# **Kaggle Steel Defect Detection using Hybrid Task Cascade**

**What is Hybrid Task Cascade?** 

Cascade is a classic yet powerful architecture that has
boosted performance on various tasks. However, how to introduce cascade to instance segmentation remains an open
question. A simple combination of Cascade **R-CNN** and
**Mask R-CNN** only brings limited gain. In exploring a more
effective approach, we find that the key to a successful instance segmentation cascade is to fully leverage the reciprocal relationship between detection and segmentation. In
this work, we propose a new framework, Hybrid Task Cascade (HTC), which differs in two important aspects: (1) instead of performing cascaded refinement on these two tasks
separately, it interweaves them for a joint multi-stage processing; (2) it adopts a fully convolutional branch to provide spatial context, which can help distinguishing hard
foreground from cluttered background. Overall, this framework can learn more discriminative features progressively
while integrating complementary features together in each
stage. Without bells and whistles, a single HTC obtains
38.4% and 1.5% improvement over a strong Cascade Mask
R-CNN baseline on MSCOCO dataset. Moreover, our overall system achieves 48.6 mask AP on the test-challenge split,
**ranking 1st in the COCO 2018 Challenge Object Detection**
Task. Code is available at: https://github.com/
open-mmlab/mmdetection.

In [0]:
%cd 
%cd ..

/root
/


In [0]:
import os
import json
import gc

import cv2
import keras
from keras import backend as K
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, load_model
from keras.layers import Input, Dropout, Conv2D, BatchNormalization, add
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras import backend as K
from keras.layers import LeakyReLU
from keras.losses import binary_crossentropy
from keras.layers.merge import concatenate, Concatenate, Add
from keras.optimizers import Adam
from keras.callbacks import Callback, ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split



Using TensorFlow backend.


# Kaggle Steel Data Download

note: Upload Kaggle.json file with your kaggle.json key to working directory before running

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls -l ~/.kaggle
!cat ~/.kaggle/kaggle.json
!pip install -q kaggle
!pip install -q kaggle-cli

total 4
-rw------- 1 root root 67 Sep  9 07:47 kaggle.json
     |████████████████████████████████| 81kB 3.8MB/s 
     |████████████████████████████████| 5.3MB 9.8MB/s 
     |████████████████████████████████| 102kB 30.2MB/s 
     |████████████████████████████████| 112kB 47.6MB/s 
     |████████████████████████████████| 51kB 13.6MB/s 


In [0]:
!kaggle competitions download -c severstal-steel-defect-detection 

 72% 5.00M/6.91M [00:00<00:00, 29.2MB/s]
100% 6.91M/6.91M [00:00<00:00, 33.9MB/s]
  0% 0.00/141k [00:00<?, ?B/s]
100% 141k/141k [00:00<00:00, 140MB/s]
 99% 1.15G/1.16G [00:09<00:00, 115MB/s]
100% 1.16G/1.16G [00:10<00:00, 123MB/s]
 94% 121M/129M [00:01<00:00, 96.9MB/s]
100% 129M/129M [00:01<00:00, 100MB/s] 


In [0]:
!mkdir data
!unzip /train_images.zip -d /data/train;
#!rm /content/train_images.zip;
!unzip /test_images.zip -d /data/test;
#!rm /content/test_images.zip;
!unzip /train.csv.zip 
#!rm /content/train.csv.zip
!cp /sample_submission.csv /data
!cp /train.csv.zip /data

Archive:  /train_images.zip
  inflating: /data/train/5e581254c.jpg  
  inflating: /data/train/fd2f7b4f4.jpg  
  inflating: /data/train/82f4c0b69.jpg  
  inflating: /data/train/18cc39190.jpg  
  inflating: /data/train/bcef3582e.jpg  
  inflating: /data/train/b1cf6d807.jpg  
  inflating: /data/train/fadccb348.jpg  
  inflating: /data/train/6bdf447c5.jpg  
  inflating: /data/train/29e57e1de.jpg  
  inflating: /data/train/0da296499.jpg  
  inflating: /data/train/049fb17a1.jpg  
  inflating: /data/train/c6bbd2e4f.jpg  
  inflating: /data/train/a4a007ea2.jpg  
  inflating: /data/train/8f5e37d0a.jpg  
  inflating: /data/train/dff1d028a.jpg  
  inflating: /data/train/6adf77c70.jpg  
  inflating: /data/train/e81ee8548.jpg  
  inflating: /data/train/440ce15f1.jpg  
  inflating: /data/train/1483bc52d.jpg  
  inflating: /data/train/4783eb121.jpg  
  inflating: /data/train/e865201fb.jpg  
  inflating: /data/train/7b90bf5c9.jpg  
  inflating: /data/train/86ded721f.jpg  
  inflating: /data/train/17e1

In [0]:
train_df = pd.read_csv('/train.csv')
train_df['ImageId'] = train_df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
train_df['ClassId'] = train_df['ImageId_ClassId'].apply(lambda x: x.split('_')[1])
train_df['hasMask'] = ~ train_df['EncodedPixels'].isna()

print(train_df.shape)
train_df.head()


(50272, 5)


,ImageId_ClassId,EncodedPixels,ImageId,ClassId,hasMask
0,0002cc93b.jpg_1,29102 12 29346 24 29602 24 29858 24 30114 24 3...,0002cc93b.jpg,1,True
1,0002cc93b.jpg_2,NaN,0002cc93b.jpg,2,False
2,0002cc93b.jpg_3,NaN,0002cc93b.jpg,3,False
3,0002cc93b.jpg_4,NaN,0002cc93b.jpg,4,False
4,00031f466.jpg_1,NaN,00031f466.jpg,1,False


In [0]:
mask_count_df = train_df.groupby('ImageId').agg(np.sum).reset_index()
mask_count_df.sort_values('hasMask', ascending=False, inplace=True)
print(mask_count_df.shape)
mask_count_df.head()

(12568, 2)


,ImageId,hasMask
10803,db4867ee8.jpg,3.0
11776,ef24da2ba.jpg,3.0
6284,7f30b9c64.jpg,2.0
9421,bf0c81db6.jpg,2.0
9615,c314f43f3.jpg,2.0


In [0]:
sub_df = pd.read_csv('/sample_submission.csv')
sub_df['ImageId'] = sub_df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
test_imgs = pd.DataFrame(sub_df['ImageId'].unique(), columns=['ImageId'])
test_imgs.head()

,ImageId
0,004f40c73.jpg
1,006f39c41.jpg
2,00b7fb703.jpg
3,00bbcd9af.jpg
4,0108ce457.jpg


In [0]:
non_missing_train_idx = mask_count_df[mask_count_df['hasMask'] > 0]
non_missing_train_idx.head()

,ImageId,hasMask
10803,db4867ee8.jpg,3.0
11776,ef24da2ba.jpg,3.0
6284,7f30b9c64.jpg,2.0
9421,bf0c81db6.jpg,2.0
9615,c314f43f3.jpg,2.0


# Build

In [0]:
# TODO: change URL to your fork of my repository if necessary.
git_repo_url = 'https://github.com/bhaktatejas922/kaggle-imaterialist.git'

# Pick the model you want to use
# Select a model in `MODELS_CONFIG`.
# Total training epochs.
total_epochs = 8

# Name of the config file.


In [0]:
import os
from os.path import exists, join, basename, splitext

project_name = os.path.abspath(splitext(basename(git_repo_url))[0])
mmdetection_dir = os.path.join(project_name, "mmdetection")
#print(mmdetection_dir)
if not exists(project_name):
    # clone "depth 1" will only get the latest copy of the relevant files.
    !git clone -q --recurse-submodules --depth 1 $git_repo_url
    # dependencies
    !pip install -q mmcv terminaltables
    # build
    !cd {mmdetection_dir} && python setup.py install


import sys
sys.path.append(mmdetection_dir)
import time
import matplotlib
import matplotlib.pylab as plt
plt.rcParams["axes.grid"] = False

     |████████████████████████████████| 51kB 2.5MB/s 
running install
running bdist_egg
running egg_info
creating mmdet.egg-info
writing mmdet.egg-info/PKG-INFO
writing dependency_links to mmdet.egg-info/dependency_links.txt
writing requirements to mmdet.egg-info/requires.txt
writing top-level names to mmdet.egg-info/top_level.txt
writing manifest file 'mmdet.egg-info/SOURCES.txt'
writing manifest file 'mmdet.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/mmdet
copying mmdet/version.py -> build/lib/mmdet
copying mmdet/__init__.py -> build/lib/mmdet
creating build/lib/mmdet/apis
copying mmdet/apis/env.py -> build/lib/mmdet/apis
copying mmdet/apis/inference.py -> build/lib/mmdet/apis
copying mmdet/apis/train.py -> build/lib/mmdet/apis
copying mmdet/apis/__init__.py -> build/lib/mmdet/apis
creating build/lib/mmdet/ops
copying mmdet/ops/__init__.py -> build/lib/mmdet/ops

In [0]:
# need to convert ImageId_ClassId to ImageId and ClassId
# and EncodedPixels to their respective classes
train_df1=train_df.dropna()
train_df1=train_df1.drop(['ImageId_ClassId', 'hasMask'], axis=1)
train_df1['Height']=256
train_df1['Width']=1600

#order it properly before zipping it 
train_df1=train_df1[['ImageId', 'EncodedPixels', 'Height', 'Width', 'ClassId']]
train_df1=train_df1.reset_index(drop=True)


In [0]:
%cd /data
train_df1.to_csv('train.csv')
!rm train.csv.zip
!zip train.csv.zip 'train.csv'
train_df1.head()

/data
  adding: train.csv (deflated 60%)


,ImageId,EncodedPixels,Height,Width,ClassId
0,0002cc93b.jpg,29102 12 29346 24 29602 24 29858 24 30114 24 3...,256,1600,1
1,0007a71bf.jpg,18661 28 18863 82 19091 110 19347 110 19603 11...,256,1600,3
2,000a4bcdd.jpg,37607 3 37858 8 38108 14 38359 20 38610 25 388...,256,1600,1
3,000f6bf48.jpg,131973 1 132228 4 132483 6 132738 8 132993 11 ...,256,1600,4
4,0014fce06.jpg,229501 11 229741 33 229981 55 230221 77 230468...,256,1600,3


In [0]:
%cd /kaggle-imaterialist/mmdetection
!bash compile.sh
!python setup.py develop

/kaggle-imaterialist/mmdetection
Building roi align op...
running build_ext
building 'roi_align_cuda' extension
creating build
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/src
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.6/dist-packages/torch/include -I/usr/local/lib/python3.6/dist-packages/torch/include/torch/csrc/api/include -I/usr/local/lib/python3.6/dist-packages/torch/include/TH -I/usr/local/lib/python3.6/dist-packages/torch/include/THC -I/usr/local/cuda/include -I/usr/include/python3.6m -c src/roi_align_cuda.cpp -o build/temp.linux-x86_64-3.6/src/roi_align_cuda.o -DTORCH_API_INCLUDE_EXTENSION_H -DTORCH_EXTENSION_NAME=roi_align_cuda -D_GLIBCXX_USE_CXX11_ABI=0 -std=c++11
/usr/local/cuda/bin/nvcc -I/usr/local/lib/python3.6/dist-packages/torch/include -I/usr/local/lib/python3.6/dist-packages/torch/include/torch/csrc/ap

In [0]:
%cd /data
sample_df=pd.read_csv('sample_submission.csv')
sample_df['ImageId'] = sample_df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
sample_df['ClassId'] = sample_df['ImageId_ClassId'].apply(lambda x: x.split('_')[1])
sample_df=sample_df.drop(['ImageId_ClassId'], axis=1)
sample_df=sample_df[['ImageId', 'EncodedPixels', 'ClassId']]

# replace sample submission with new one to work with mmdetection
!rm /data/sample_submission.csv
sample_df.to_csv('sample_submission.csv')

print(train_df.shape)
sample_df.head()

/data
(50272, 5)


,ImageId,EncodedPixels,ClassId
0,004f40c73.jpg,1 1,1
1,004f40c73.jpg,1 1,2
2,004f40c73.jpg,1 1,3
3,004f40c73.jpg,1 1,4
4,006f39c41.jpg,1 1,1


In [0]:
%cd /kaggle-imaterialist/scrips
!bash prepare_weights.sh


/kaggle-imaterialist/scrips
--2019-09-09 07:52:08--  https://s3.ap-northeast-2.amazonaws.com/open-mmlab/mmdetection/models/htc/htc_dconv_c3-c5_mstrain_400_1400_x101_64x4d_fpn_20e_20190408-0e50669c.pth
Resolving s3.ap-northeast-2.amazonaws.com (s3.ap-northeast-2.amazonaws.com)... 52.219.60.4
Connecting to s3.ap-northeast-2.amazonaws.com (s3.ap-northeast-2.amazonaws.com)|52.219.60.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 574412463 (548M) [application/x-www-form-urlencoded]
Saving to: ‘/dumps/htc_dconv_c3-c5_mstrain_400_1400_x101_64x4d_fpn_20e_20190408-0e50669c.pth’

/dumps/htc_dconv_c3 100%[===================>] 547.80M  17.5MB/s    in 32s     

2019-09-09 07:52:40 (17.3 MB/s) - ‘/dumps/htc_dconv_c3-c5_mstrain_400_1400_x101_64x4d_fpn_20e_20190408-0e50669c.pth’ saved [574412463/574412463]



In [0]:
#to do before runninging this is above
%cd /kaggle-imaterialist/scrips
!pip install iterative-stratification

!bash create_mmdetection_train.sh
!bash create_mmdetection_test.sh
!bash split.sh

/kaggle-imaterialist/scrips
7095 6666
100% 6666/6666 [00:24<00:00, 271.44it/s]
7204 1801
100% 1801/1801 [00:07<00:00, 241.36it/s]
train size: 6599, val size: 67


Copying data and mounting drive for storage

In [0]:
!rm -r /data_temp

rm: cannot remove '/data_temp': No such file or directory


In [0]:
!cp -r /data /data_temp
%mkdir /data/data
!cp -a /data_temp/. /data/data



In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# get trained model from google drive
!cp -r /content/gdrive/My\ Drive/checkpoint/dumps /dumps

#Training

In [0]:
import torch
%cd /kaggle-imaterialist/scrips
#!CUDA_VISIBLE_DEVICES=['/device:GPU:0'] bash dist_train.sh /kaggle-imaterialist/configs/htc_dconv_c3-c5_mstrain_x101_64x4d_fpn_20e_1200x1900.py --validate 
!python /kaggle-imaterialist/mmdetection/tools/train.py /kaggle-imaterialist/configs/htc_dconv_c3-c5_mstrain_x101_64x4d_fpn_20e_1200x1900.py --gpus 1 --validate --resume_from /dumps/htc_dconv_c3-c5_mstrain_400_1400_x101_64x4d_fpn_20e_20190408-0e50669c_prune.pth

/kaggle-imaterialist/scrips
2019-09-09 08:06:52,721 - INFO - Distributed training: False
2019-09-09 08:06:59,586 - INFO - load checkpoint from /dumps/htc_dconv_c3-c5_mstrain_400_1400_x101_64x4d_fpn_20e_20190408-0e50669c_prune.pth
2019-09-09 08:06:59,989 - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: semantic_head.lateral_convs.0.conv.weight, semantic_head.lateral_convs.0.conv.bias, semantic_head.lateral_convs.1.conv.weight, semantic_head.lateral_convs.1.conv.bias, semantic_head.lateral_convs.2.conv.weight, semantic_head.lateral_convs.2.conv.bias, semantic_head.lateral_convs.3.conv.weight, semantic_head.lateral_convs.3.conv.bias, semantic_head.lateral_convs.4.conv.weight, semantic_head.lateral_convs.4.conv.bias, semantic_head.convs.0.conv.weight, semantic_head.convs.0.conv.bias, semantic_head.convs.1.conv.weight, semantic_head.convs.1.conv.bias, semantic_head.convs.2.conv.weight, semantic_head.convs.2.conv.bias, semantic_head.convs

In [0]:
!python /kaggle-imaterialist/mmdetection/tools/test.py /kaggle-imaterialist/configs/htc_dconv_c3-c5_mstrain_x101_64x4d_fpn_20e_1200x1900.py /dumps/htc_dconv_c3-c5_mstrain_400_1400_x101_64x4d_fpn_20e_20190408-0e50669c_prune.pth --out /data/data/test_mmdetection.pkl #--eval segm 
# output path is /data/data/test_mmdetection.pkl

The model and loaded state dict do not match exactly

unexpected key in source state_dict: semantic_head.lateral_convs.0.conv.weight, semantic_head.lateral_convs.0.conv.bias, semantic_head.lateral_convs.1.conv.weight, semantic_head.lateral_convs.1.conv.bias, semantic_head.lateral_convs.2.conv.weight, semantic_head.lateral_convs.2.conv.bias, semantic_head.lateral_convs.3.conv.weight, semantic_head.lateral_convs.3.conv.bias, semantic_head.lateral_convs.4.conv.weight, semantic_head.lateral_convs.4.conv.bias, semantic_head.convs.0.conv.weight, semantic_head.convs.0.conv.bias, semantic_head.convs.1.conv.weight, semantic_head.convs.1.conv.bias, semantic_head.convs.2.conv.weight, semantic_head.convs.2.conv.bias, semantic_head.convs.3.conv.weight, semantic_head.convs.3.conv.bias, semantic_head.conv_embedding.conv.weight, semantic_head.conv_embedding.conv.bias, semantic_head.conv_logits.weight, semantic_head.conv_logits.bias

missing keys in source state_dict: mask_head.0.convs.1.conv.bias, bbo

In [0]:
!cp -r /data/data/test_mmdetection.pkl /content/gdrive/My\ Drive/test

In [0]:
import pickle
file = open('/content/gdrive/My Drive/test/test_mmdetection.pkl', 'rb')
data=pickle.load(file)
file.close()
for item in data:
  if item!=data[0]:
    print(item)

9/4/19

Try to change config file to reduce epochs, change augmentation size, etc. 


In [0]:
!python /kaggle-imaterialist/mmdetection/tools/analyze_logs.py plot_curve log1.json log2.json --keys bbox_mAP --legend run1 run2


In [0]:
#!cp -r /dumps /content/gdrive/My\ Drive/checkpoint

In [0]:
#!rm -r /dumps

In [0]:
#!rm -r /dumps /content/gdrive/My\ Drive/checkpoint

In [0]:
#!CUDA_VISIBLE_DEVICES= bash dist_test.sh /kaggle-imaterialist/configs/htc_dconv_c3-c5_mstrain_x101_64x4d_fpn_20e_1200x1900.py --nproc_per_node 1


look  at commands in the .sh file below 

needs redo 

https://github.com/bhaktatejas922/kaggle-imaterialist/blob/master/scrips/dist_test_ensemble.sh

# Testing

In [0]:
!cd /data
!mv sample_submission.csv test_ensemble_submission.csv
!cd /kaggle-imaterialist/scrips
!CUDA_VISIBLE_DEVICES= bash /kaggle-imaterialist/scrips/dist_test_ensemble.sh /kaggle-imaterialist/configs/htc_dconv_c3-c5_mstrain_x101_64x4d_fpn_20e_1200x1900.py 0 --validate 


mv: cannot stat 'sample_submission.csv': No such file or directory
Traceback (most recent call last):
  File "/kaggle-imaterialist/src/submit.py", line 113, in <module>
    main()
  File "/kaggle-imaterialist/src/submit.py", line 92, in main
    predictions = mmcv.load(args.predictions)
  File "/usr/local/lib/python3.6/dist-packages/mmcv/fileio/io.py", line 40, in load
    obj = handler.load_from_path(file, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/mmcv/fileio/handlers/pickle_handler.py", line 13, in load_from_path
    filepath, mode='rb', **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/mmcv/fileio/handlers/base.py", line 21, in load_from_path
    with open(filepath, mode) as f:
FileNotFoundError: [Errno 2] No such file or directory: '/data/test_ensemble_predictions.pkl'


**Currently testing with max epochs set to 5**